# PROJECT_VIVINO

In [16]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session, sessionmaker, registry

# open vivino.db from data directory
vivino_engine = create_engine('sqlite:///../data/vivino.db', echo=True)

# establish and instantiate session as vivino_session
vivino_session = sessionmaker(bind=vivino_engine)
vivino_session = vivino_session()

# create a registry for the vivino_session
mapper_registry = registry()
Base = mapper_registry.generate_base()




In [17]:
# query schema for all tables in vivino.db
for table in tablenames_list:
    query_schema = vivino_engine.execute(
        f"PRAGMA table_info({table});"
        )
    print(f"\n{table} schema:")
    for row in query_schema:
        print(row)


2023-08-30 14:45:14,737 INFO sqlalchemy.engine.Engine PRAGMA table_info(countries);
2023-08-30 14:45:14,740 INFO sqlalchemy.engine.Engine [raw sql] ()

countries schema:
(0, 'code', 'VARCHAR', 1, None, 1)
(1, 'name', 'VARCHAR', 0, None, 0)
(2, 'regions_count', 'INTEGER', 0, None, 0)
(3, 'users_count', 'INTEGER', 0, None, 0)
(4, 'wines_count', 'INTEGER', 0, None, 0)
(5, 'wineries_count', 'INTEGER', 0, None, 0)
2023-08-30 14:45:14,742 INFO sqlalchemy.engine.Engine PRAGMA table_info(grapes);
2023-08-30 14:45:14,743 INFO sqlalchemy.engine.Engine [raw sql] ()

grapes schema:
(0, 'id', 'INTEGER', 1, None, 1)
(1, 'name', 'VARCHAR', 0, None, 0)
2023-08-30 14:45:14,746 INFO sqlalchemy.engine.Engine PRAGMA table_info(wineries);
2023-08-30 14:45:14,746 INFO sqlalchemy.engine.Engine [raw sql] ()

wineries schema:
(0, 'id', 'INTEGER', 1, None, 1)
(1, 'name', 'VARCHAR', 0, None, 0)
2023-08-30 14:45:14,749 INFO sqlalchemy.engine.Engine PRAGMA table_info(flavor_groups);
2023-08-30 14:45:14,750 INFO sq

## QUESTION 1

We want to highlight 10 wines to increase our sales. Which ones should we choose and why?
- Those with good ratings but not a big amount of ratings
- Natural wines?

#### CATEGORY 1 - BEST RETAIL WINE

In [3]:
## join countries, region, and wines tables to create df_wines
query_wines = vivino_engine.execute(
    """SELECT wines.id AS wine_id,
    wines.name AS wine_name,
    wines.is_natural AS is_natural,
    wines.ratings_average,
    wines.ratings_count,
    regions.name AS region_name,
    countries.name AS country_name
    FROM wines
    LEFT JOIN regions ON wines.region_id = regions.id
    LEFT JOIN countries ON regions.country_code = countries.code
    WHERE wines.ratings_average >= 4.5
    AND wines.ratings_count >= 650
    AND wines.is_natural = 1;
    """
)

# fetch query_wines
fetch_query_wines = query_wines.fetchall()

# initialize fetch_query_wines into pandas dataframe named df_wines
df_wines = pd.DataFrame(
    fetch_query_wines,
    columns=query_wines.keys()
)

# display df_wines
display(df_wines)


2023-08-30 13:53:58,045 INFO sqlalchemy.engine.Engine SELECT wines.id AS wine_id,
    wines.name AS wine_name,
    wines.is_natural AS is_natural,
    wines.ratings_average,
    wines.ratings_count,
    regions.name AS region_name,
    countries.name AS country_name
    FROM wines
    LEFT JOIN regions ON wines.region_id = regions.id
    LEFT JOIN countries ON regions.country_code = countries.code
    WHERE wines.ratings_average >= 4.5
    AND wines.ratings_count >= 650
    AND wines.is_natural = 1;
    
2023-08-30 13:53:58,046 INFO sqlalchemy.engine.Engine [raw sql] ()


,wine_id,wine_name,is_natural,ratings_average,ratings_count,region_name,country_name
0,1101360,Le Bourg Saumur Champigny,1,4.5,1264,Saumur-Champigny,France
1,1101361,Les Poyeux Saumur Champigny,1,4.5,2005,Saumur-Champigny,France
2,1219509,Camí Pesseroles,1,4.5,1083,Priorat,Espagne
3,1627127,Magma,1,4.5,748,Terre Siciliane,Italie
4,1680438,Il San Lorenzo Bianco,1,4.5,673,Marche,Italie


We can recommend the wines listed above because those wines are natural, European Wines substantiated with a solid customer reviews and rating.

#### CATEGORY 2 - BEST VINTAGE WINE

In [4]:
# query vintage wines
query_vintages= vivino_engine.execute("""SELECT avg(rank) AS avg_rank,
vintages.name as vintage_wine_name,
vintages.wine_id AS vintage_wine_id,
vintages.ratings_average AS ratings_average,
vintages.ratings_count AS ratings_count,
vintages.price_euros AS retail_price_euros,
vintages.bottle_volume_ml AS bottle_volume_ml,
vintage_toplists_rankings.rank AS rank,
vintage_toplists_rankings.previous_rank AS previous_rank,
regions.name AS region_name,
countries.name AS country_name
FROM vintages
LEFT JOIN wines ON vintages.wine_id = wines.id
LEFT JOIN regions ON wines.region_id = regions.id
LEFT JOIN countries ON regions.country_code = countries.code
INNER JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
WHERE vintages.ratings_count >= 3000
    AND vintages.ratings_average >= 4.5
    AND vintage_toplists_rankings.rank = vintage_toplists_rankings.previous_rank
    AND (vintage_toplists_rankings.rank <= 10 OR vintage_toplists_rankings.previous_rank <= 10)
GROUP BY vintages.wine_id 
ORDER BY avg_rank ASC;
""")

# initialize best_vintages_df
fetch_query_vintages = query_vintages.fetchall()

best_vintages_df = pd.DataFrame(
    fetch_query_vintages,
    columns=query_vintages.keys()
)

# drop 'price_discounted_from', 'price_discount_percentage', 'id', 'name',

# display best_vintages_df
display(best_vintages_df)



2023-08-30 13:53:58,074 INFO sqlalchemy.engine.Engine SELECT avg(rank) AS avg_rank,
vintages.name as vintage_wine_name,
vintages.wine_id AS vintage_wine_id,
vintages.ratings_average AS ratings_average,
vintages.ratings_count AS ratings_count,
vintages.price_euros AS retail_price_euros,
vintages.bottle_volume_ml AS bottle_volume_ml,
vintage_toplists_rankings.rank AS rank,
vintage_toplists_rankings.previous_rank AS previous_rank,
regions.name AS region_name,
countries.name AS country_name
FROM vintages
LEFT JOIN wines ON vintages.wine_id = wines.id
LEFT JOIN regions ON wines.region_id = regions.id
LEFT JOIN countries ON regions.country_code = countries.code
INNER JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
WHERE vintages.ratings_count >= 3000
    AND vintages.ratings_average >= 4.5
    AND vintage_toplists_rankings.rank = vintage_toplists_rankings.previous_rank
    AND (vintage_toplists_rankings.rank <= 10 OR vintage_toplists_rankings.previous_ran

,avg_rank,vintage_wine_name,vintage_wine_id,ratings_average,ratings_count,retail_price_euros,bottle_volume_ml,rank,previous_rank,region_name,country_name
0,5.5,Château Pontet-Canet Pauillac (Grand Cru Class...,14362,4.7,3275,597.5,750,5,5,Pauillac,France
1,6.0,Krug Rosé,79632,4.6,3605,349.0,750,6,6,Champagne,France
2,6.0,Krug Grande Cuvée,7122486,4.6,28513,245.0,750,6,6,Champagne,France
3,9.5,Laurent-Perrier Grand Siècle Champagne (Grande...,1238419,4.5,9824,183.7,750,9,9,Champagne,France
4,10.0,Antinori Tignanello 2016,1652,4.6,13709,420.0,750,10,10,Toscana,Italie


We recommend these vintage wines due to the fact that they have been consistently marking their place in our vintages_toplists_rankings table throughout multiple years, and their positions are substantiated with a significant amount of ratings and overall average rating score. 

## QUESTION 2
We have a marketing budget for this year. Which country should we prioritise and why?
- Those with high user count?
- those we low user count?
- Those with a upcoming user popularity

In [5]:
# CRITERIA 1 - COUNTRIES WITH HIGH USER COUNTS

## query countries table to yield top 5 countries based on user counts

query_top_5_user_count = vivino_engine.execute(
    """SELECT countries.name AS country_name,
    countries.users_count AS user_count
    FROM countries
    ORDER BY user_count DESC
    LIMIT 5;
    """
)

## initialize top_5_user_count_df
fetch_query_top_5_user_count = query_top_5_user_count.fetchall()

top_5_user_count_df = pd.DataFrame(
    fetch_query_top_5_user_count,
    columns=query_top_5_user_count.keys()
)

## display top_5_user_count_df
display(top_5_user_count_df)

2023-08-30 13:53:58,097 INFO sqlalchemy.engine.Engine SELECT countries.name AS country_name,
    countries.users_count AS user_count
    FROM countries
    ORDER BY user_count DESC
    LIMIT 5;
    
2023-08-30 13:53:58,098 INFO sqlalchemy.engine.Engine [raw sql] ()


,country_name,user_count
0,États-Unis,12273684
1,France,5973301
2,Italie,4270717
3,Allemagne,2549989
4,Espagne,2264396


In [6]:
# CRITERIA 3 - COUNTRIES WITH UPCOMING USER POPULARITY

## initializing query to yield upcoming popular countries
query_upcoming_popular_country = vivino_engine.execute(
    """SELECT countries.name AS country_name,
    SUM(wines.ratings_count) AS ratings_count
    FROM wines
    LEFT JOIN regions ON wines.region_id = regions.id
    LEFT JOIN countries ON regions.country_code = countries.code
    GROUP BY country_name
    ORDER BY ratings_count DESC;
"""
)

## initialize upcoming_popular_country_df
fetch_query_upcoming_popular_country = query_upcoming_popular_country.fetchall()

upcoming_popular_country_df = pd.DataFrame(
    fetch_query_upcoming_popular_country,
    columns=query_upcoming_popular_country.keys()
)

## display upcoming_popular_country_df
display(upcoming_popular_country_df)


2023-08-30 13:53:58,113 INFO sqlalchemy.engine.Engine SELECT countries.name AS country_name,
    SUM(wines.ratings_count) AS ratings_count
    FROM wines
    LEFT JOIN regions ON wines.region_id = regions.id
    LEFT JOIN countries ON regions.country_code = countries.code
    GROUP BY country_name
    ORDER BY ratings_count DESC;

2023-08-30 13:53:58,115 INFO sqlalchemy.engine.Engine [raw sql] ()


,country_name,ratings_count
0,Italie,2135839
1,France,2124809
2,États-Unis,834263
3,Espagne,746710
4,Argentine,283673
5,Chili,159256
6,Portugal,62454
7,Afrique du Sud,59995
8,Australie,52138
9,Hongrie,30564


Based on the information above, we suggest to focus the marketing resources on establishing customer base in these following countries 
 1. Italy (listed as Italie)
 2. France (listed as France)
 3. Spain (listed as Espagne)
 4. United States of America (listed as États-Unis)
 5. Germany (listed as Allemagne)

 The first 3 countries are countries from which the wines are highly sought after by customers in general on the basis of customer engagement toward the wines the respective countries, which is measured through ratings count. On top of that, they are also countries with a substantial amount of customers as well. 
 The last 2 countries are countries from which the wines are gaining more popularity. However, they are countries with a significant amount of customers who uses vivino. 
 

## QUESTION 3

We have detected that a big cluster of customers like a specific combination of tastes.
We have identified a few primary keywords that match this.
We would like you to find all the wines that have those keywords. 
To ensure the accuracy of our selection, ensure that more than 10 users confirmed those keywords.
 Also, identify the group_name related to those keywords. 
 - coffee
- toast
- green apple
- cream
- citrus

In [7]:
# joining wines, keywords_wine , flavor_groups, keywords, regions, country tables in order to create keyword_flavor_profile_df for each wine in wines table with aggregated keywords per flavor_group
query_keyword_flavor_profile = vivino_engine.execute(
    """
    SELECT wines.id AS wine_id,
        wines.name AS wine_name,
        keywords.name AS flavor_keyword,
        flavor_groups.name AS flavor_group,
        keywords_wine.keyword_type AS keyword_type,
        regions.name AS region_name,
        countries.name AS country_name,
        wines.ratings_average AS ratings_average,
        wines.ratings_count AS ratings_count
    FROM wines
    LEFT JOIN keywords_wine ON wines.id = keywords_wine.wine_id
    LEFT JOIN keywords ON keywords_wine.keyword_id = keywords.id
    LEFT JOIN flavor_groups ON keywords_wine.group_name = flavor_groups.name
    LEFT JOIN regions ON wines.region_id = regions.id
    LEFT JOIN countries ON regions.country_code = countries.code
    WHERE keywords_wine.count >= 10
        AND keyword_type = 'primary'
    ORDER BY ratings_count DESC;
    """
)

## initialize keyword_flavor_profile_df
fetch_query_keyword_flavor_profile = query_keyword_flavor_profile.fetchall()

keyword_flavor_profile_df = pd.DataFrame(
    fetch_query_keyword_flavor_profile,
    columns=query_keyword_flavor_profile.keys()
)

## group flavor_keyword per flavor_group
keyword_flavor_profile_df = keyword_flavor_profile_df.groupby(
    [
        'wine_id', 
        'wine_name', 
        'flavor_group', 
        'keyword_type', 
        'region_name', 
        'country_name', 
        'ratings_average', 
        'ratings_count'
        ])['flavor_keyword'].apply(', '.join).reset_index()


## display keyword_flavor_profile_df
display(keyword_flavor_profile_df)



2023-08-30 13:53:58,136 INFO sqlalchemy.engine.Engine 
    SELECT wines.id AS wine_id,
        wines.name AS wine_name,
        keywords.name AS flavor_keyword,
        flavor_groups.name AS flavor_group,
        keywords_wine.keyword_type AS keyword_type,
        regions.name AS region_name,
        countries.name AS country_name,
        wines.ratings_average AS ratings_average,
        wines.ratings_count AS ratings_count
    FROM wines
    LEFT JOIN keywords_wine ON wines.id = keywords_wine.wine_id
    LEFT JOIN keywords ON keywords_wine.keyword_id = keywords.id
    LEFT JOIN flavor_groups ON keywords_wine.group_name = flavor_groups.name
    LEFT JOIN regions ON wines.region_id = regions.id
    LEFT JOIN countries ON regions.country_code = countries.code
    WHERE keywords_wine.count >= 10
        AND keyword_type = 'primary'
    ORDER BY ratings_count DESC;
    
2023-08-30 13:53:58,136 INFO sqlalchemy.engine.Engine [raw sql] ()


,wine_id,wine_name,flavor_group,keyword_type,region_name,country_name,ratings_average,ratings_count,flavor_keyword
0,1213,Vin Santo di Montepulciano,dried_fruit,primary,Vin Santo di Montepulciano,Italie,4.6,773,"raisin, fig"
1,1213,Vin Santo di Montepulciano,earth,primary,Vin Santo di Montepulciano,Italie,4.6,773,honey
2,1213,Vin Santo di Montepulciano,oak,primary,Vin Santo di Montepulciano,Italie,4.6,773,"caramel, chocolate, coffee"
3,1268,St. Henri Shiraz,black_fruit,primary,South Australia,Australie,4.4,17326,"blackberry, plum, black fruit, blackcurrant, b..."
4,1268,St. Henri Shiraz,dried_fruit,primary,South Australia,Australie,4.4,17326,"prune, raisin, dried fruit, fig"
...,...,...,...,...,...,...,...,...,...
4775,9954550,Tinto,black_fruit,primary,Ribera del Duero,Espagne,4.5,585,blackberry
4776,9954550,Tinto,oak,primary,Ribera del Duero,Espagne,4.5,585,oak
4777,9954550,Tinto,red_fruit,primary,Ribera del Duero,Espagne,4.5,585,cherry
4778,9957200,802.A Superblend,oak,primary,South Australia,Australie,4.5,66,oak


In [15]:
# Identifying the group_name of the requested flavor keywords
query_requested_keywords = vivino_engine.execute(
    """
    SELECT count(wines.name) as keyword_match_counter, 
        keywords_wine.wine_id,
        wines.name AS wine_name,
        keywords_wine.keyword_type AS keyword_type,
        keywords_wine.group_name AS flavor_group
    FROM keywords_wine 
    JOIN wines on wine_id = wines.id 
    WHERE keywords_wine.count > 10 
        AND keyword_id IN (
            SELECT id 
            FROM keywords 
            WHERE name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
    )
        AND keyword_type = 'primary' 
        GROUP BY wine_id 
        ORDER BY keyword_match_counter DESC
        LIMIT 10;
    """
)

## INitialize keyword_flavor_profile_df
fetch_query_requested_keywords = query_requested_keywords.fetchall()

requested_keyword_df = pd.DataFrame(
    fetch_query_requested_keywords,
    columns=query_requested_keywords.keys()
)

## display keyword_flavor_profile_df
display(requested_keyword_df)


2023-08-30 14:41:17,675 INFO sqlalchemy.engine.Engine 
    SELECT count(wines.name) as keyword_match_counter, 
        keywords_wine.wine_id,
        wines.name AS wine_name,
        keywords_wine.keyword_type AS keyword_type,
        keywords_wine.group_name AS flavor_group
    FROM keywords_wine 
    JOIN wines on wine_id = wines.id 
    WHERE keywords_wine.count > 10 
        AND keyword_id IN (
            SELECT id 
            FROM keywords 
            WHERE name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
    )
        AND keyword_type = 'primary' 
        GROUP BY wine_id 
        ORDER BY keyword_match_counter DESC
        LIMIT 10;
    
2023-08-30 14:41:17,677 INFO sqlalchemy.engine.Engine [raw sql] ()


,keyword_match_counter,wine_id,wine_name,keyword_type,flavor_group
0,5,7122486,Grande Cuvée,primary,citrus_fruit
1,5,3630450,Cuvée des Enchanteleurs Brut Champagne,primary,citrus_fruit
2,5,3102815,P2 Plénitude Brut Champagne,primary,citrus_fruit
3,5,1692390,Sir Winston Churchill Brut Champagne,primary,citrus_fruit
4,5,1441099,Blanc des Millénaires,primary,citrus_fruit
5,5,1192720,Dom Ruinart Blanc de Blancs Brut Champagne,primary,citrus_fruit
6,5,1164702,R.D Extra Brut Champagne (Récemment Dégorgé),primary,citrus_fruit
7,5,1153863,Sauternes,primary,citrus_fruit
8,5,1136950,Comtes de Champagne Blanc de Blancs,primary,citrus_fruit
9,5,1127349,Le Mesnil Blanc de Blancs (Cuvée S) Brut Champ...,primary,citrus_fruit


 - 'keyword_flavor_profile_df' highlights the 'flavor_group' of each associated 'flavor_key' per 'wine_name'. 

 - 'requested_keyword_df' highlights the 'flavor_group' name in which the requested 'flavor_key' belong to. 
     - 'keyword_match_counter' column acts as an indicator that suggests the matched 'wine_name' fulfills all of the requested 'flavor_key'
     - The requested 'flavor_key' are: 'coffee', 'toast', 'green apple', 'cream', 'citrus'
     - The requested 'flavor_key' belongs to 'citrus_fruit' 'flavor_group'


## QUESTION 4
We would like to select wines that are easy to find all over the world. 
1. Find the top 3 most common grape all over the world 
2. For each grape, give us the the 5 best rated wines.

In [59]:
# 1. Find the top 3 most common grape all over the world 

query_4 = vivino_engine.execute(
    """
    SELECT
        grapes.name AS grape_name,
        count(most_used_grapes_per_country.country_code) AS num_available_countries,
        most_used_grapes_per_country.wines_count
    FROM
        most_used_grapes_per_country
    LEFT JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
    LEFT JOIN countries ON most_used_grapes_per_country.country_code = countries.code
    WHERE
        most_used_grapes_per_country.wines_count > 100000
    GROUP BY most_used_grapes_per_country.grape_id
    ORDER BY
        num_available_countries DESC
    LIMIT 3;
    """
)

## Fetch query_4 and initialize into df_most_common_grapes
fetch_query_4 = query_4.fetchall()
df_most_common_grapes = pd.DataFrame(
    fetch_query_4,
    columns=query_4.keys()
)

## display df_most_common_grapes
display(df_most_common_grapes)



2023-08-30 15:41:55,453 INFO sqlalchemy.engine.Engine 
    SELECT
        grapes.name AS grape_name,
        count(most_used_grapes_per_country.country_code) AS num_available_countries,
        most_used_grapes_per_country.wines_count
    FROM
        most_used_grapes_per_country
    LEFT JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
    LEFT JOIN countries ON most_used_grapes_per_country.country_code = countries.code
    WHERE
        most_used_grapes_per_country.wines_count > 100000
    GROUP BY most_used_grapes_per_country.grape_id
    ORDER BY
        num_available_countries DESC
    LIMIT 3;
    
2023-08-30 15:41:55,454 INFO sqlalchemy.engine.Engine [raw sql] ()


,grape_name,num_available_countries,wines_count
0,Cabernet Sauvignon,12,801751
1,Merlot,11,566719
2,Chardonnay,6,604208


Based on the information above, here are the top 3 grape varieties based on their usage in different wines throughout the world
 1. Cabernet Sauvignon
 2. Merlot
 3. Chardonnay

In [72]:
# Churning 5 best rated wines per df_most_common_grapes['grape_name']
top_grapes = df_most_common_grapes['grape_name'].tolist()

## query top 5 cabernet sauvignon wines
query_cabernet_sauvignon = vivino_engine.execute(f"""
    SELECT
        wines.name,
        wines.ratings_average AS ranks,
        wines.ratings_count AS review_amount,
        most_used_grapes_per_country.grape_id AS grapes_id,
        grapes.name AS grapes_name,
        regions.name AS region_name,
        countries.name AS country_name
    FROM
        wines
    JOIN regions ON wines.region_id = regions.id
    JOIN countries ON regions.country_code = countries.code
    JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
    LEFT JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE
        grapes.name = '{top_grapes[0]}'
        AND wines.ratings_average >= 4.5
    ORDER BY review_amount DESC
    LIMIT 5;
    """
)

## Fetch query_cabernet_sauvignon and initialize into df_cabernet_sauvignon
fetch_query_cabernet_sauvignon = query_cabernet_sauvignon.fetchall()
df_cabernet_sauvignon = pd.DataFrame(
    fetch_query_cabernet_sauvignon,
    columns=query_cabernet_sauvignon.keys()
)

## display df_cabernet_sauvignon
display(df_cabernet_sauvignon)

## query top 5 merlot wines
query_merlot =  vivino_engine.execute(f"""
    SELECT
        wines.name,
        wines.ratings_average AS ranks,
        wines.ratings_count AS review_amount,
        most_used_grapes_per_country.grape_id AS grapes_id,
        grapes.name AS grapes_name,
        regions.name AS region_name,
        countries.name AS country_name
    FROM
        wines
    JOIN regions ON wines.region_id = regions.id
    JOIN countries ON regions.country_code = countries.code
    JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
    LEFT JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE
        grapes.name = '{top_grapes[1]}'
        AND wines.ratings_average >= 4.5
    ORDER BY review_amount DESC
    LIMIT 5;
    """
)

## Fetch query_merlot and initialize into df_merlot
fetch_query_merlot = query_merlot.fetchall()
df_merlot = pd.DataFrame(
    fetch_query_merlot,
    columns=query_merlot.keys()
)

## display df_merlot
display(df_merlot)

## query top 5 Chardonnay wines
query_chardonnay =  vivino_engine.execute(f"""
    SELECT
        wines.name,
        wines.ratings_average AS ranks,
        wines.ratings_count AS review_amount,
        most_used_grapes_per_country.grape_id AS grapes_id,
        grapes.name AS grapes_name,
        regions.name AS region_name,
        countries.name AS country_name
    FROM
        wines
    JOIN regions ON wines.region_id = regions.id
    JOIN countries ON regions.country_code = countries.code
    JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
    LEFT JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE
        grapes.name = '{top_grapes[2]}'
        AND wines.ratings_average >= 4.5
    ORDER BY review_amount DESC
    LIMIT 5;
    """
)

## Fetch query_chardonnay and initialize into df_chardonnay
fetch_query_chardonnay = query_chardonnay.fetchall()
df_chardonnay = pd.DataFrame(
    fetch_query_chardonnay,
    columns=query_chardonnay.keys()
)

## display df_chardonnay
display(df_chardonnay)

    







# ## Initialize grape_name_df
# fetch_query_grape_name = query_grape_name.fetchall()
# grape_name_df = pd.DataFrame(
#     fetch_query_grape_name,
#     columns=query_grape_name.keys()
# )

# ## display grape_name_df
# display(grape_name_df)


2023-08-30 16:08:48,664 INFO sqlalchemy.engine.Engine 
    SELECT
        wines.name,
        wines.ratings_average AS ranks,
        wines.ratings_count AS review_amount,
        most_used_grapes_per_country.grape_id AS grapes_id,
        grapes.name AS grapes_name,
        regions.name AS region_name,
        countries.name AS country_name
    FROM
        wines
    JOIN regions ON wines.region_id = regions.id
    JOIN countries ON regions.country_code = countries.code
    JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
    LEFT JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE
        grapes.name = 'Cabernet Sauvignon'
        AND wines.ratings_average >= 4.5
    ORDER BY review_amount DESC
    LIMIT 5;
    
2023-08-30 16:08:48,666 INFO sqlalchemy.engine.Engine [raw sql] ()


,name,ranks,review_amount,grapes_id,grapes_name,region_name,country_name
0,Cabernet Sauvignon,4.6,157944,2,Cabernet Sauvignon,Napa Valley,États-Unis
1,Opus One,4.6,77053,2,Cabernet Sauvignon,Napa Valley,États-Unis
2,Valbuena 5º,4.6,72784,2,Cabernet Sauvignon,Ribera del Duero,Espagne
3,Ribera del Duero,4.5,69278,2,Cabernet Sauvignon,Ribera del Duero,Espagne
4,Almaviva,4.6,48075,2,Cabernet Sauvignon,Puente Alto,Chili


2023-08-30 16:08:48,678 INFO sqlalchemy.engine.Engine 
    SELECT
        wines.name,
        wines.ratings_average AS ranks,
        wines.ratings_count AS review_amount,
        most_used_grapes_per_country.grape_id AS grapes_id,
        grapes.name AS grapes_name,
        regions.name AS region_name,
        countries.name AS country_name
    FROM
        wines
    JOIN regions ON wines.region_id = regions.id
    JOIN countries ON regions.country_code = countries.code
    JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
    LEFT JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE
        grapes.name = 'Merlot'
        AND wines.ratings_average >= 4.5
    ORDER BY review_amount DESC
    LIMIT 5;
    
2023-08-30 16:08:48,679 INFO sqlalchemy.engine.Engine [raw sql] ()


,name,ranks,review_amount,grapes_id,grapes_name,region_name,country_name
0,Brut Champagne,4.6,146377,10,Merlot,Champagne,France
1,Tignanello,4.5,142513,10,Merlot,Toscana,Italie
2,Sassicaia,4.6,107646,10,Merlot,Bolgheri Sassicaia,Italie
3,60 Sessantanni Old Vines Primitivo di Manduria,4.5,94289,10,Merlot,Primitivo di Manduria,Italie
4,Pauillac (Premier Grand Cru Classé),4.6,73615,10,Merlot,Pauillac,France


2023-08-30 16:08:48,690 INFO sqlalchemy.engine.Engine 
    SELECT
        wines.name,
        wines.ratings_average AS ranks,
        wines.ratings_count AS review_amount,
        most_used_grapes_per_country.grape_id AS grapes_id,
        grapes.name AS grapes_name,
        regions.name AS region_name,
        countries.name AS country_name
    FROM
        wines
    JOIN regions ON wines.region_id = regions.id
    JOIN countries ON regions.country_code = countries.code
    JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
    LEFT JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE
        grapes.name = 'Chardonnay'
        AND wines.ratings_average >= 4.5
    ORDER BY review_amount DESC
    LIMIT 5;
    
2023-08-30 16:08:48,691 INFO sqlalchemy.engine.Engine [raw sql] ()


,name,ranks,review_amount,grapes_id,grapes_name,region_name,country_name
0,Cabernet Sauvignon,4.6,157944,5,Chardonnay,Napa Valley,États-Unis
1,Brut Champagne,4.6,146377,5,Chardonnay,Champagne,France
2,Tignanello,4.5,142513,5,Chardonnay,Toscana,Italie
3,Sassicaia,4.6,107646,5,Chardonnay,Bolgheri Sassicaia,Italie
4,60 Sessantanni Old Vines Primitivo di Manduria,4.5,94289,5,Chardonnay,Primitivo di Manduria,Italie


The information above highlights 5 best-rated wines for each grape variety based on their overall 'ranks' and 'review_amount'

For those who likes wines using Cabernet Sauvignon grape variety, we are happy to recommend these wines
 1. Cabernet Sauvignon - Napa Valley - USA
 2. Opus One - Napa Valley - USA
 3. Valbuena 5º - Ribera del Duero - Spain
 4. Ribera del Duero - Ribera del Duero - Spain
 5. Almaviva - Puente Alto - Chili